In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import SGD
from scipy.integrate import quad, nquad

## Set hyperparameters

In [ ]:
dim = 1

input_dim = dim
hidden_dim = 4 * input_dim
output_dim = 1

activation_func = "relu"

## Implement loss function

In [ ]:
lam = 1.0  # Tbc
rho = 1.0  # Tbc

def f(x):
    return 1

def h_i(x, i):
    return x ** 2  # Tbc

def g(x):
    return x ** 2  # tbc

def mu_i_density(x, i):
    return np.exp(-x**2 / 2) / np.sqrt(2 * np.pi)  # tbc

def sample_mu(n_samples, dim):
    return tf.random.normal((n_samples, dim))  # tbc

def beta_gamma():
    return 1

def cost_function(x, y):
    return x+y

def theta(x, y):
    return 1

def phi_theta_gamma(samples):
    h_integral = tf.reduce_sum(tf.map_fn(lambda i: tf.reduce_mean(h_i(samples[:, i], i)), tf.range(dim), dtype=tf.float32))
    g_integral = tf.reduce_mean(tf.map_fn(g, samples, dtype=tf.float32))

    # y_values = tf.random.normal((10 ** 4, dim))  # TODO: Not explicit n_samples
    # theta_integral = tf.constant(0.0, dtype=tf.float32)
    # for x in samples:
    #     for y in y_values:
    #         integral_value += beta_gamma(f(y) - tf.reduce_sum([h_i(y, i) for i in tf.range(dim)]) - lam * cost_function(x, y) - g(x)) * theta(x, y)

    theta_integral = 0

    return lam * rho + h_integral + g_integral + theta_integral

class CustomLoss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    
    def call(self, x_true, x_pred):
        #samples = sample_mu(tf.shape(x_pred)[0], dim)
        return phi_theta_gamma(x_pred)


## Create model

In [ ]:
initilizer = RandomNormal(mean=0.0, stddev=1.0)

model = Sequential()
model.add(Input(shape=(input_dim,)))
model.add(Dense(units=hidden_dim, activation=activation_func, kernel_initializer=initilizer))
model.add(Dense(units=hidden_dim, activation=activation_func, kernel_initializer=initilizer))
model.add(Dense(units=hidden_dim, activation=activation_func, kernel_initializer=initilizer))
model.add(Dense(units=output_dim, kernel_initializer=initilizer))

In [ ]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000, decay_rate=0.9
)

opt = keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(optimizer=opt, loss=CustomLoss())

model.summary()

In [ ]:
n_samples = 10 ** 4
x_samples = sample_mu(n_samples, dim).numpy()
x_true_irrelevant = np.zeros_like(x_samples)

model.fit(x_samples, x_true_irrelevant, epochs=5, batch_size=32)

In [ ]:
x_test = tf.linspace(-10, 10, 500)
x_test = tf.reshape(x_test, (-1, 1))
y_test = model.predict(x_test)

print(x_test, y_test)

plt.plot(x_test, y_test, label='Approximierte Funktion')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.legend()
plt.show()

# Not needed

In [ ]:
""" import numpy as np
from scipy.optimize import minimize
from scipy.integrate import quad, nquad

# Beispiel für \rho
rho = 1.0  # Anpassen je nach Definition
lambda_val = 1.0

#Dummy-Beispiel für h_i und g. Muss später per NN optimiert werden (?).
def h_i(x, i):
    return x**2

def g(x):
    return sum(x)

# Beispiel für \beta_{\gamma} Funktion
def beta_gamma(z, gamma):
    # Dummy-Beispiel für die Funktion, anpassen je nach Definition
    return np.exp(-gamma * z**2)

# Dummy-Funktion für Verteilungen \bar{\mu}_i und \bar{\mu}
def mu_i_density(y, i):
    # Dummy-Dichtefunktion, anpassen je nach Definition
    return np.exp(-y**2 / 2) / np.sqrt(2 * np.pi)

def mu_density(x):
    # Dummy-Dichtefunktion, anpassen je nach Definition
    return np.exp(-np.sum(x**2) / 2) / (2 * np.pi)**(len(x)/2)

# Kostenfunktion c(x, y)
def cost_function(x, y):
    # Beispiel-Kostenfunktion, anpassen je nach Definition
    return np.linalg.norm(x - y)

# Funktion \phi_{\theta,\gamma}(f)
def phi_theta_gamma(f, theta, gamma, d):

    # Integrale berechnen
    h_integral = sum(quad(lambda y: h_i(y, i) * mu_i_density(y, i), -np.inf, np.inf)[0] for i in range(d))
    g_integral = nquad(lambda *x: g(x) * mu_density(np.array(x)), [(-np.inf, np.inf)] * d)[0]

    def integrand(x, y):
            return beta_gamma(f(y) - sum(h_i(y[0], i) for i in range(d)) - lambda_val * cost_function(x, y) - g, gamma) * theta(x, y)

    beta_integral = nquad(integrand, [(-np.inf, np.inf)] * (2*d))

    return lambda_val * rho + h_integral + g_integral + beta_integral


    # # Initiale Parameter
    # initial_params = np.zeros(d + 1 + d)
    
    # # Optimierung
    # result = minimize(objective, initial_params, method='SLSQP')
    
    # return result.fun


# Beispielaufruf der Funktion
if __name__ == "__main__":
    # Dummy \theta Funktion
    def theta(x, y):
        return 1.0  # Dummy-Wert, anpassen je nach Definition

    # Dummy f Funktion
    def f(y):
        return np.sum(y**2)  # Dummy-Wert, anpassen je nach Definition

    # Parameter
    gamma = 1.0
    d = 1  # Dimension

    # Funktionsaufruf
    result = phi_theta_gamma(f, theta, gamma, d)
    print(f'Result: {result}')
 """

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Beispiel-Optimierungsproblem: Minimierung von (x - 2)^2
# Ziel ist es, eine Funktion f(x) zu approximieren, die dieses Problem minimiert.

# Erstellen Sie einige Beispiel-Daten (unüberwachtes Lernen)
x_train = np.random.rand(1000, 1) * 4 - 2  # Werte zwischen -2 und 2

# Definieren Sie die Struktur des neuronalen Netzes
model2 = Sequential([
    Dense(64, activation='relu', input_shape=(1,)),
    Dense(64, activation='relu'),
    Dense(1)
])

# Definieren Sie die benutzerdefinierte Verlustfunktion
def custom_loss(y_true, y_pred):
    return tf.reduce_mean((y_pred - 2) ** 2)  # Beispiel: (f(x) - 2)^2

# Kompilieren Sie das Modell mit der benutzerdefinierten Verlustfunktion
model2.compile(optimizer=Adam(learning_rate=0.01), loss=custom_loss)

# Da es sich um unüberwachtes Lernen handelt, können wir Dummy-Zielwerte verwenden (werden nicht wirklich verwendet)
y_dummy = np.zeros_like(x_train)

# Trainieren Sie das Modell
model2.fit(x_train, y_dummy, epochs=50, batch_size=32)

# Testen Sie das Modell
x_test = np.linspace(-2, 2, 100).reshape(-1, 1)
y_pred = model2.predict(x_test)

import matplotlib.pyplot as plt

plt.plot(x_test, y_pred, label='Approximierte Funktion')
plt.axhline(2, color='r', linestyle='--', label='Zielwert 2')
plt.xlabel('x')
plt.ylabel('f(x)')
plt.legend()
plt.show()
